# Preprocessing 01 - Data Preparation - Convert EEG .fif files to .set and .vhdr files
## From replication package (Peitek et al.)

## Import libraries

In [ ]:
import os
import mne
from pybv import write_brainvision
import numpy as np
import regex as re
from tqdm.notebook import tqdm

## Set Input directory, Path to Montage, and Channels to drop

In [ ]:
filepath_out: str = "./data/eeg_tmp/raw"
montage_path = "./data/EEG/AC-64.bvef"
channels_to_drop = ['x_dir', 'y_dir', 'z_dir']

## Helper Functions For Transorming EEG .fif files to .set and .vhdr files

In [ ]:
def load_fif_file(filename, filepath):
    full_filename = os.path.join(filepath, filename)
    return mne.io.read_raw_fif(fname=full_filename + '.fif', preload=True)


def convert_fif_to_set(filename, load_from, save_to, number):
    # load fif file
    raw = load_fif_file(filename, load_from)

    # load and set montage
    montage = mne.channels.read_custom_montage(montage_path, head_size=0.085)
    raw.set_montage(montage)

    # drop channels that wont be used
    raw.drop_channels(channels_to_drop)

    # save data to eeglab format
    savepath: str = os.path.join(save_to, filename + "_" + number + ".set")
    mne.export.export_raw(savepath, raw, 'eeglab', 'auto', overwrite=True, verbose="ERROR")

    # transform events to eeglab format
    events = raw.info["events"]
    np_e = []
    for e in events:
        np_e.append([e['list'][0], e['list'][2]])

    np_array = np.asarray(np_e)

    # save events to eeglab format
    channels = raw.ch_names
    df = raw.get_data(channels)
    filename = filename + "_" + number
    write_brainvision(data=df, sfreq=500, ch_names=channels, folder_out=filepath_out, fname_base=filename, events=np_array, overwrite=True)

## Transform EEG .fif files to .set and .vhdr files

In [ ]:
# get all folders in data/filteredData
folders = [f.path for f in os.scandir('./data/rawData') if f.is_dir()]
# extract the numbers
folders = [f.split('/')[-1] for f in folders]
numbers = [int(re.findall(r'\d+', f)[0]) for f in folders]
# numbers to str with leading zeros
numbers = [str(n).zfill(1) for n in numbers]



# iterate over all folders
for number in tqdm(numbers):
    print("-----------------------------------------------------------------------------------------")
    print("Participant " + number)

    if os.path.exists("./data/filteredData/Participant" + number + "/filtered_data.csv"):
        print("Skipped: Already processed")
        continue

    # construct path to folder
    load_from = "./data/rawData/Participant" + number
    save_to = "./data/eeg_tmp/raw"
    file_name = "eeg_raw"
    # transform file via side effect
    convert_fif_to_set(file_name, load_from, save_to, number)
